In [ ]:
# %% JUPYTER CELL: Run SSL DINOv2 on selected samples (e.g. 37–110)

from pathlib import Path
import os, glob, os.path as op
import sys

# ─────────────────────────────────────────────
# 1) Detect USER_BASE_DIR (AAU / SDU style)
# ─────────────────────────────────────────────
def _detect_user_base():
    aau = glob.glob("/work/Member Files:*")
    if aau:
        return op.basename(aau[0])        # e.g. "Member Files: MatiasMose#8097"
    sdu = [d for d in glob.glob("/work/*#*") if op.isdir(d)]
    return op.basename(sdu[0]) if sdu else None

USER_BASE_DIR = os.environ.get("USER_BASE_DIR") or _detect_user_base()
if not USER_BASE_DIR:
    raise RuntimeError("Could not determine USER_BASE_DIR")
os.environ["USER_BASE_DIR"] = USER_BASE_DIR

WORK_ROOT = Path("/work") / USER_BASE_DIR

# ─────────────────────────────────────────────
# 2) Configure SSL data root + sample range
# ─────────────────────────────────────────────
# Root folder where all "Sample XX" live – same as supervised:
SSL_TRAINING_ROOT = WORK_ROOT / "CellScanData" / "Zoom10x - Quality Assessment_Cleaned"

# 🔧 EDIT THESE TWO ONLY when you change SSL range:
SSL_FIRST_SAMPLE = 37
SSL_LAST_SAMPLE  = 110

if SSL_FIRST_SAMPLE > SSL_LAST_SAMPLE:
    raise ValueError(f"SSL_FIRST_SAMPLE ({SSL_FIRST_SAMPLE}) > SSL_LAST_SAMPLE ({SSL_LAST_SAMPLE})")

# Pass range + root to LightlyTrainSSL via env vars
os.environ["SSL_TRAINING_ROOT"] = SSL_TRAINING_ROOT.as_posix()
os.environ["SSL_FIRST_SAMPLE"] = str(SSL_FIRST_SAMPLE)
os.environ["SSL_LAST_SAMPLE"]  = str(SSL_LAST_SAMPLE)

print("USER_BASE_DIR:", USER_BASE_DIR)
print("SSL_TRAINING_ROOT:", SSL_TRAINING_ROOT)
print(f"SSL sample range: {SSL_FIRST_SAMPLE}–{SSL_LAST_SAMPLE}")

# ─────────────────────────────────────────────
# 3) Optional: sanity check which Sample folders exist
# ─────────────────────────────────────────────
existing_samples = []
missing_samples = []

for i in range(SSL_FIRST_SAMPLE, SSL_LAST_SAMPLE + 1):
    d = SSL_TRAINING_ROOT / f"Sample {i}"
    if d.is_dir():
        existing_samples.append(i)
    else:
        missing_samples.append(i)

print(f"Found {len(existing_samples)} Sample folders in range.")
if missing_samples:
    print("⚠️ Missing Sample folders (will simply contribute 0 images):", missing_samples)

# ─────────────────────────────────────────────
# 4) Run SSL training via LightlyTrainSSL.py
# ─────────────────────────────────────────────
%cd /work/projects/myproj

!{sys.executable} -u LightlyTrainSSL.py
